# Airline Sentiment Analysis

## Table of Contents:
* [Data Collection](#DataCollection)
* [Data Organization](#DataOrganization)
* [Data Definition](#DataDefinition)
* [Data Cleaning](#DataCleaning)

In [1]:
#imports
import pandas as pd
#this will deal with punctuation
import string
import re
from string import digits
import nltk #using the Natural Language Toolkit
from nltk.corpus import stopwords
#Lemmatization
from nltk.stem import WordNetLemmatizer

# 1. Data Collection <a class="anchor" id="DataCollection"></a>

In [2]:
#get data
tweets = pd.read_csv("../Data/Tweets.csv")

# 2. Data Organization <a class="anchor" id="DataOrganization"></a>

In [3]:
#view the heading of data
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [4]:
#check shape
tweets.shape

(14640, 15)

In [5]:
#view summary of data
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [6]:
#check for null values
tweets.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [7]:
#percentage of null values
((tweets.isnull().sum() * 100)/ len(tweets.index))

tweet_id                         0.000000
airline_sentiment                0.000000
airline_sentiment_confidence     0.000000
negativereason                  37.308743
negativereason_confidence       28.128415
airline                          0.000000
airline_sentiment_gold          99.726776
name                             0.000000
negativereason_gold             99.781421
retweet_count                    0.000000
text                             0.000000
tweet_coord                     93.039617
tweet_created                    0.000000
tweet_location                  32.329235
user_timezone                   32.923497
dtype: float64

"airline_sentiment_gold" , "negativereason_gold", and "tweet_coord" columns are missing over 90% of their input. I will drop them

In [8]:
tweets = tweets.drop(columns=["airline_sentiment_gold" , "negativereason_gold", "tweet_coord"])
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [9]:
#check for more null values
print(tweets.isnull().sum())

print(((tweets.isnull().sum() * 100)/ len(tweets.index)))

tweet_id                           0
airline_sentiment                  0
airline_sentiment_confidence       0
negativereason                  5462
negativereason_confidence       4118
airline                            0
name                               0
retweet_count                      0
text                               0
tweet_created                      0
tweet_location                  4733
user_timezone                   4820
dtype: int64
tweet_id                         0.000000
airline_sentiment                0.000000
airline_sentiment_confidence     0.000000
negativereason                  37.308743
negativereason_confidence       28.128415
airline                          0.000000
name                             0.000000
retweet_count                    0.000000
text                             0.000000
tweet_created                    0.000000
tweet_location                  32.329235
user_timezone                   32.923497
dtype: float64


In [10]:
#view negativereason column
tweets[(tweets["negativereason"].isnull()) & (tweets["airline_sentiment"] == "negative")]

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone


### The columns that are null from the "negativereason" column is because the "airline_sentiment" column is neutral or positive.

Let's view the negativereason_confidence column

In [11]:
#negativereason_confidence 
tweets[(tweets["negativereason_confidence"].isnull())  & (tweets["airline_sentiment"] == "negative")]

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone


### The columns that are null from the "negativereason_confidence" column is because the "airline_sentiment" column is neutral or positive.

In [12]:
#tweet_location
tweets[(tweets["tweet_location"].isnull())].head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,negative,1.0000,Can't Tell,0.6842,Virgin America,jnardino,0,@VirginAmerica seriously would pay $30 a fligh...,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)


In [13]:
tweets[(tweets["tweet_location"].isnull())].groupby("airline_sentiment").size()

airline_sentiment
negative    3142
neutral      962
positive     629
dtype: int64

Most of the null tweet location are from negative sentiments.

### The columns that are null from the "tweet_location" column is because the user more than likely do not have their locations on.

Below is a view of all of the user_timezone column that is null.

In [14]:
#view null user_timezone
tweets[tweets["user_timezone"].isnull()].groupby("airline_sentiment").size()

airline_sentiment
negative    3170
neutral      971
positive     679
dtype: int64

In [15]:
#user_timezone > where the user timezone is null and the tweet location isn't null 
tzTweet = tweets[(tweets["user_timezone"].isnull()) & (tweets["tweet_location"].notnull())]

tzTweet.groupby("airline_sentiment").size()

airline_sentiment
negative    1362
neutral      430
positive     357
dtype: int64

In [16]:
#user_timezone > where the user timezone is null and the tweet location is null 
timezoneTweet = tweets[(tweets["user_timezone"].isnull()) & (tweets["tweet_location"].isnull())]

timezoneTweet.groupby("airline_sentiment").size()

airline_sentiment
negative    1808
neutral      541
positive     322
dtype: int64

# 3. Data Definition <a class="anchor" id="DataDefinition"></a>

                    
 1   airline_sentiment              
 2   airline_sentiment_confidence  
 3   negativereason                 
 4   negativereason_confidence    
 5   airline                        
 6   airline_sentiment_gold         
 7   name                          
 8   negativereason_gold            
 9   retweet_count                 
 10  text                           
 11  tweet_coord                  
 12  tweet_created                 
 13  tweet_location                
 14  user_timezone   

# 4. Data Cleaning <a class="anchor" id="DataCleaning"></a>

The following needs to be done to preprocess the data:

1. Make text lowercase
2. Removing punctuations, URLs, names
3. Tokenization
4. Removing stopwords ("this", "is", etc.)
4. Stemming

### Lowercase
Converting the "text" column to lowercase to help with preprocessing.

In [17]:
tweets["text"] = tweets["text"].str.lower()
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@virginamerica what @dhepburn said.,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@virginamerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,@virginamerica i didn't today... must mean i n...,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@virginamerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@virginamerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### Remove numbers

In [18]:
#remove numbers
tweets["text"] = tweets["text"].str.replace('\d+', '', regex=True)
tweets["text"].head()

0                  @virginamerica what @dhepburn said.
1    @virginamerica plus you've added commercials t...
2    @virginamerica i didn't today... must mean i n...
3    @virginamerica it's really aggressive to blast...
4    @virginamerica and it's a really big bad thing...
Name: text, dtype: object

### Punctuation

In [19]:
def remove_punc(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct = ''.join(no_punct)
    return words_wo_punct

In [20]:
#remove all of the punctuations
tweets["text"] = tweets["text"].apply(lambda x: remove_punc(x))
tweets["text"].head()

0                     virginamerica what dhepburn said
1    virginamerica plus youve added commercials to ...
2    virginamerica i didnt today must mean i need t...
3    virginamerica its really aggressive to blast o...
4    virginamerica and its a really big bad thing a...
Name: text, dtype: object

### Remove whitespace

In [21]:
#REMOVE WHITESPACE
tweets["text"] = tweets["text"].str.strip()
tweets["text"].head()

0                     virginamerica what dhepburn said
1    virginamerica plus youve added commercials to ...
2    virginamerica i didnt today must mean i need t...
3    virginamerica its really aggressive to blast o...
4    virginamerica and its a really big bad thing a...
Name: text, dtype: object

### Tokenization

Split word to make a list

In [22]:
def token(text):
    split=re.split("\W", text)
    return split

tweets['text'] = tweets["text"].apply(lambda x: token(x))
tweets["text"].head()

0                [virginamerica, what, dhepburn, said]
1    [virginamerica, plus, youve, added, commercial...
2    [virginamerica, i, didnt, today, must, mean, i...
3    [virginamerica, its, really, aggressive, to, b...
4    [virginamerica, and, its, a, really, big, bad,...
Name: text, dtype: object

### Stopwords

In [23]:
def remove_stopwords(text):
    words = [word for word in text if word not in stopwords.words('english')]
    return words

In [24]:
tweets["text"] = tweets["text"].apply(lambda x: remove_stopwords(x))
tweets["text"].head()

0                      [virginamerica, dhepburn, said]
1    [virginamerica, plus, youve, added, commercial...
2    [virginamerica, didnt, today, must, mean, need...
3    [virginamerica, really, aggressive, blast, obn...
4             [virginamerica, really, big, bad, thing]
Name: text, dtype: object

### Lemmatizing

Reduce word to its root form.

In [25]:
#lemmatizer
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(x) for x in text]
    return lem_text

In [26]:
tweets["text"] = tweets["text"].apply(lambda x: word_lemmatizer(x))
tweets["text"].head()

0                      [virginamerica, dhepburn, said]
1    [virginamerica, plus, youve, added, commercial...
2    [virginamerica, didnt, today, must, mean, need...
3    [virginamerica, really, aggressive, blast, obn...
4             [virginamerica, really, big, bad, thing]
Name: text, dtype: object

### Langdetect

In [27]:
"""from langdetect import detect

#turn into string object
tweets["text"] = tweets["text"].to_string()
tweets['lang'] = tweets["text"].apply(detect)"""

'from langdetect import detect\n\n#turn into string object\ntweets["text"] = tweets["text"].to_string()\ntweets[\'lang\'] = tweets["text"].apply(detect)'

In [28]:
#tweets[tweets["lang"] != 'en'].head()

All of the tweets are in english

Emotion sensor features: https://www.kaggle.com/iwilldoit/emotions-sensor-data-setamount of repetition

spell checking

-emojis - convert to text version

Text length

% caps > 50%

Look through the data yourself to see if you find any patterns that are indicative of a given class, make 
features out of those

Semantic Role Labeller: https://cogcomp.seas.upenn.edu/page/software_view/Curator

### Remove Emojis

In [29]:
"""import emoji

tweets["text"] = tweets["text"].apply(lambda x: emoji.demojize(x))
tweets["text"].head()"""

'import emoji\n\ntweets["text"] = tweets["text"].apply(lambda x: emoji.demojize(x))\ntweets["text"].head()'

In [30]:
"""tweets["text"] = tweets["text"].to_string() 

tweets['text'] = tweets["text"].apply(lambda x: token(x))
tweets["text"].head()"""

'tweets["text"] = tweets["text"].to_string() \n\ntweets[\'text\'] = tweets["text"].apply(lambda x: token(x))\ntweets["text"].head()'

In [31]:
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,"[virginamerica, dhepburn, said]",2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,"[virginamerica, plus, youve, added, commercial...",2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,"[virginamerica, didnt, today, must, mean, need...",2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,"[virginamerica, really, aggressive, blast, obn...",2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,"[virginamerica, really, big, bad, thing]",2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


### spellcheck

In [32]:
#df.two.apply(lambda txt: ''.join(textblob.TextBlob(txt).correct()))

In [33]:
#tweets["text"].head(10)

In [34]:
"""tweets["text"] = tweets["text"].apply(lambda x: ''.join(textblob.TextBlob(x).correct()))
tweets["text"].head(10)"""

'tweets["text"] = tweets["text"].apply(lambda x: \'\'.join(textblob.TextBlob(x).correct()))\ntweets["text"].head(10)'

### Emotion sensor

In [35]:
#!pip install text2emotion

In [36]:
#import text2emotion as te
#tweets["emotion"] = te.get_emotion(tweets["text"])

In [37]:
#pip install text2emotion

### Vectorization

Try CountVec and Tfidf and choose which scores better on Logistic Regression/baseline model

Make sure to pick min_df > 3 (at the very least) 

You can visualize document frequency by top words/n_grams and see at what min_df the words/phrases stop being sensible/interesting (Casey’s Preprocessing) 

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split

### CountVec

In [55]:
tweets["text"].dtypes

dtype('O')

In [53]:
tweets["text"] = tweets["text"].astype(str)

In [65]:
#train test split
cv = CountVectorizer(min_df=0.50)
X = cv.fit_transform(tweets['text'])
y = tweets['airline_sentiment'].values.astype('int64')

X_train, X_test, y_train, y_test = train_test_split(X, y)

ValueError: invalid literal for int() with base 10: 'neutral'

#### get base model with Logistic Regression

In [ ]:
logReg = LogisticRegression().fit(X_train, y_train)
#print accuracy

print("LR Accuracy: %0.2f%%" % (100 * logReg.score(X_test, y_test)))

training_accuracy = logReg.score(X_train, y_train)
test_accuracy = logReg.score(X_test, y_test)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))


### Tfidf

In [64]:
td = TfidfVectorizer(max_df=0.95)
X = td.fit_transform(tweets['text'])
y = tweets['airline_sentiment'].values.astype(np.int)

X_train, X_test, y_train, y_test = train_test_split(X, y)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [ ]:
logReg = LogisticRegression().fit(X_train, y_train)
#print accuracy

print("LR Accuracy: %0.2f%%" % (100 * logReg.score(X_test, y_test)))

training_accuracy = logReg.score(X_train, y_train)
test_accuracy = logReg.score(X_test, y_test)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))

In [ ]:
"""# TfidfVectorizer 
# CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
# set of documents
train = ['The sky is blue.','The sun is bright.']
test = ['The sun in the sky is bright', 'We can see the shining sun, the bright sun.']
# instantiate the vectorizer object
countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')
# convert th documents into a matrix
count_wm = countvectorizer.fit_transform(train)
tfidf_wm = tfidfvectorizer.fit_transform(train)
#retrieve the terms found in the corpora
# if we take same parameters on both Classes(CountVectorizer and TfidfVectorizer) , it will give same output of get_feature_names() methods)
#count_tokens = tfidfvectorizer.get_feature_names() # no difference
count_tokens = countvectorizer.get_feature_names()
tfidf_tokens = tfidfvectorizer.get_feature_names()
df_countvect = pd.DataFrame(data = count_wm.toarray(),index = ['Doc1','Doc2'],columns = count_tokens)
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)
print("Count Vectorizer\n")
print(df_countvect)
print("\nTD-IDF Vectorizer\n")
print(df_tfidfvect)"""

In [ ]:
"""#your turn
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y)
clf = MultinomialNB().fit(xtrain, ytrain)
print("MN Accuracy: %0.2f%%" % (100 * clf.score(xtest, ytest))
      
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print("Accuracy on training data: %0.2f" % (training_accuracy))
print("Accuracy on test data:     %0.2f" % (test_accuracy))"""